Date: 2016-12-19

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
%qtconsole
import os
import sys
import collections
import itertools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sys.path.append(os.path.join(os.path.abspath(os.path.pardir), 'src'))
import data_processing
import ripple_decoding
import ripple_detection
import analysis
import spectral

In [16]:
sampling_frequency = 1500
Animal = collections.namedtuple('Animal', {'directory', 'short_name'})
animals = {
    'HPa': Animal(directory='HPa_direct', short_name='HPa'),
    'HPb': Animal(directory='HPb_direct', short_name='HPb'),
    'HPc': Animal(directory='HPc_direct', short_name='HPc')
}
ripple_covariates = ['is_spike', 'session_time',
                     'ripple_trajectory', 'ripple_direction']

In [6]:
epoch_index1 = ('HPa', 1, 4)
ripple_times1 = ripple_detection.get_epoch_ripples(
    epoch_index1, animals, sampling_frequency=sampling_frequency)
ripple_info1 = ripple_decoding.decode_ripple(
    epoch_index1, animals, ripple_times1)[0]


Detecting ripples for Animal HPa, Day 1, Epoch #4...



In [12]:
epoch_index2 = ('HPa', 1, 6)
ripple_times2 = ripple_detection.get_epoch_ripples(
    epoch_index2, animals, sampling_frequency=sampling_frequency)
ripple_info2 = ripple_decoding.decode_ripple(
    epoch_index2, animals, ripple_times2)[0]


Detecting ripples for Animal HPa, Day 1, Epoch #6...


Decoding ripples for Animal HPa, Day 1, Epoch #6:
                                                   area numspikes
animal day epoch_ind tetrode_number neuron_number                
HPa    1   6         1              1               CA1       949
                                    2               CA1      3756
                                    3               CA1       791
                                    4               CA1       430
                                    5               CA1      3455
                                    6               CA1      1262
                     4              1               CA1       445
                                    2               CA1       490
                                    3               CA1      1153
                                    4               CA1       385
                                    5               CA1      1319
                                    

/Users/edeno/anaconda3/envs/Jadhav-2016-Data-Analysis/lib/python3.5/site-packages/statsmodels/genmod/families/links.py:422: RuntimeWarning: overflow encountered in exp
  return np.exp(z)
/Users/edeno/anaconda3/envs/Jadhav-2016-Data-Analysis/lib/python3.5/site-packages/statsmodels/genmod/families/family.py:347: RuntimeWarning: divide by zero encountered in true_divide
  endog_mu = self._clean(endog/mu)
/Users/edeno/anaconda3/envs/Jadhav-2016-Data-Analysis/lib/python3.5/site-packages/statsmodels/genmod/families/family.py:347: RuntimeWarning: invalid value encountered in true_divide
  endog_mu = self._clean(endog/mu)
/Users/edeno/anaconda3/envs/Jadhav-2016-Data-Analysis/lib/python3.5/site-packages/statsmodels/genmod/families/family.py:121: RuntimeWarning: invalid value encountered in multiply
  return 1. / (self.link.deriv(mu)**2 * self.variance(mu))
/Users/edeno/anaconda3/envs/Jadhav-2016-Data-Analysis/lib/python3.5/site-packages/statsmodels/genmod/families/family.py:121: RuntimeWarning:

		...Neuron #19
		...Neuron #20
		...Neuron #21
		...Neuron #22
	Fitting state transition model...


/Users/edeno/anaconda3/envs/Jadhav-2016-Data-Analysis/lib/python3.5/site-packages/numpy/lib/function_base.py:804: RuntimeWarning: invalid value encountered in greater_equal
  not_smaller_than_edge = (sample[:, i] >= edges[i][-1])


	Setting initial conditions...
	Decoding ripples...


In [17]:
ripple_info1

,ripple_trajectory,ripple_direction,ripple_state_probability,ripple_start_time,ripple_end_time,number_of_unique_neurons_spiking,number_of_spikes,session_time,is_spike
ripple_number,,,,,,,,,
1,outbound,reverse,0.498917,5868.199267,5868.372600,5,8,early,isSpike
2,inbound,forward,0.633105,5872.819267,5872.927933,3,3,early,isSpike
3,inbound,reverse,0.281745,5873.846600,5873.899267,0,0,early,noSpike
4,inbound,reverse,0.402418,5873.950600,5874.049267,1,1,early,isSpike
5,inbound,reverse,0.276688,5874.483267,5874.521933,0,0,early,noSpike
6,inbound,forward,0.517409,5878.823933,5878.910600,1,2,early,isSpike
7,inbound,reverse,0.286399,5882.324600,5882.399933,0,0,early,noSpike
8,inbound,forward,0.397855,5886.320600,5886.468600,4,5,early,isSpike
9,inbound,forward,0.511196,5893.720600,5893.879267,4,9,early,isSpike


In [18]:
ripple_info2

,ripple_trajectory,ripple_direction,ripple_state_probability,ripple_start_time,ripple_end_time,number_of_unique_neurons_spiking,number_of_spikes,session_time,is_spike
ripple_number,,,,,,,,,
1,inbound,reverse,0.595940,7921.666633,7921.745967,2,4,early,isSpike
2,outbound,reverse,0.561937,7921.877300,7921.964633,2,4,early,isSpike
3,inbound,forward,0.547585,7922.491967,7922.665300,5,8,early,isSpike
4,outbound,forward,0.338473,7933.180633,7933.286633,3,3,early,isSpike
5,outbound,reverse,0.606531,7937.411300,7937.616633,4,12,early,isSpike
6,outbound,reverse,0.402329,7937.970633,7938.059967,3,4,early,isSpike
7,outbound,reverse,0.360470,7939.431300,7939.548633,2,4,early,isSpike
8,inbound,forward,0.424844,7952.981967,7953.149300,5,8,early,isSpike
9,inbound,reverse,0.414657,7953.693967,7953.763300,1,1,early,isSpike


In [32]:
grouped1 = ripple_info1.groupby('ripple_trajectory')
grouped2 = ripple_info2.groupby('ripple_trajectory')

In [33]:
list(itertools.combinations(grouped1.groups.keys(), 2))

[('outbound', 'inbound')]

In [34]:
list(itertools.combinations(grouped2.groups.keys(), 2))

[('outbound', 'inbound')]

In [37]:
gamma_frequency_medFreqRes1 = dict(
    sampling_frequency=sampling_frequency,
    time_window_duration=0.100,
    time_window_step=0.100,
    desired_frequencies=(15, 125),
    time_halfbandwidth_product=1,
    window_of_interest=(-0.500, 0.400)
)

analysis.coherence_by_ripple_type(epoch_index1, animals, ripple_info1, 'ripple_trajectory',
                             coherence_name='coherence', multitaper_params=gamma_frequency_medFreqRes1)

                                     area  depth descrip
animal day epoch_ind tetrode_number                     
HPa    1   4         1                CA1    114  riptet
                     2                CA1    124     NaN
                     4                CA1    117  riptet
                     5                CA1    119  riptet
                     6                CA1    110  riptet
                     7                CA1    111  riptet
                     8               iCA1    116  riptet
                     9               iCA1    100  riptet
                     10              iCA1     99     NaN
                     11              iCA1    110  riptet
                     12              iCA1    118  riptet
                     13              iCA1    120     NaN
                     14              iCA1    106  riptet
                     15               PFC     96     NaN
                     16               PFC     94     NaN
                     17        

In [38]:
analysis.coherence_by_ripple_type(epoch_index2, animals, ripple_info2, 'ripple_trajectory',
                             coherence_name='coherence', multitaper_params=gamma_frequency_medFreqRes1)

                                     area  depth descrip
animal day epoch_ind tetrode_number                     
HPa    1   6         1                CA1    114  riptet
                     2                CA1    124     NaN
                     4                CA1    117  riptet
                     5                CA1    119  riptet
                     6                CA1    110  riptet
                     7                CA1    111  riptet
                     8               iCA1    116  riptet
                     9               iCA1    100  riptet
                     10              iCA1     99     NaN
                     11              iCA1    110  riptet
                     12              iCA1    118  riptet
                     13              iCA1    120     NaN
                     14              iCA1    106  riptet
                     15               PFC     96     NaN
                     16               PFC     94     NaN
                     17        

In [44]:
store1 = pd.HDFStore('../Processed-Data/HPa_01_04.h5')
store2 = pd.HDFStore('../Processed-Data/HPa_01_06.h5')

In [45]:
k = store1.keys()
store1.close()
k

['/coherence/tetrode_info',
 '/coherence/tetrode_pair_info',
 '/coherence/tetrode0019_tetrode0020/ripple_trajectory/inbound',
 '/coherence/tetrode0019_tetrode0020/ripple_trajectory/inbound_outbound',
 '/coherence/tetrode0019_tetrode0020/ripple_trajectory/outbound',
 '/coherence/tetrode0018_tetrode0020/ripple_trajectory/inbound',
 '/coherence/tetrode0018_tetrode0020/ripple_trajectory/inbound_outbound',
 '/coherence/tetrode0018_tetrode0020/ripple_trajectory/outbound',
 '/coherence/tetrode0018_tetrode0019/ripple_trajectory/inbound',
 '/coherence/tetrode0018_tetrode0019/ripple_trajectory/inbound_outbound',
 '/coherence/tetrode0018_tetrode0019/ripple_trajectory/outbound',
 '/coherence/tetrode0017_tetrode0020/ripple_trajectory/inbound',
 '/coherence/tetrode0017_tetrode0020/ripple_trajectory/inbound_outbound',
 '/coherence/tetrode0017_tetrode0020/ripple_trajectory/outbound',
 '/coherence/tetrode0017_tetrode0019/ripple_trajectory/inbound',
 '/coherence/tetrode0017_tetrode0019/ripple_trajectory

In [46]:
k = store2.keys()
store2.close()
k

['/coherence/tetrode_info',
 '/coherence/tetrode_pair_info',
 '/coherence/tetrode0019_tetrode0020/ripple_trajectory/inbound',
 '/coherence/tetrode0019_tetrode0020/ripple_trajectory/inbound_outbound',
 '/coherence/tetrode0019_tetrode0020/ripple_trajectory/outbound',
 '/coherence/tetrode0018_tetrode0020/ripple_trajectory/inbound',
 '/coherence/tetrode0018_tetrode0020/ripple_trajectory/inbound_outbound',
 '/coherence/tetrode0018_tetrode0020/ripple_trajectory/outbound',
 '/coherence/tetrode0018_tetrode0019/ripple_trajectory/inbound',
 '/coherence/tetrode0018_tetrode0019/ripple_trajectory/inbound_outbound',
 '/coherence/tetrode0018_tetrode0019/ripple_trajectory/outbound',
 '/coherence/tetrode0017_tetrode0020/ripple_trajectory/inbound',
 '/coherence/tetrode0017_tetrode0020/ripple_trajectory/inbound_outbound',
 '/coherence/tetrode0017_tetrode0020/ripple_trajectory/outbound',
 '/coherence/tetrode0017_tetrode0019/ripple_trajectory/inbound',
 '/coherence/tetrode0017_tetrode0019/ripple_trajectory